In [1]:
import json
import re

In [2]:
# This file contains 8600ish users Data
# In the form: {'MBTI Type', 'Social Media Posts'}
dataFile = open('formatted_data.json', 'r')

In [3]:
data = []

In [4]:
# loading the json data into a list
for line in dataFile:
    data.append(json.loads(line))

In [5]:
types = []
posts = []

In [6]:
# Separating data into 2 separate lists for preprocessing
# For the most part we process the 'post' data
# we skip the first element as that is only the label
for i in range(1, len(data)):
    types.append(data[i]['Type'])
    posts.append(data[i]['Post'])

In [7]:
import pickle

tokenized_posts = []
with open ('tokenized_formatted_data.txt', 'rb') as fp:
    tokenized_posts = pickle.load(fp)

In [8]:
all_words_list = []

for user in tokenized_posts:
    for word in user:
        all_words_list.append(word)


In [9]:
from collections import Counter

In [10]:
freq_list = Counter(all_words_list)
dictionary = freq_list.most_common(10000)

In [11]:
dictionary = list(zip(*dictionary))[0]

In [12]:
nums = range(0, 10000)
word_int = dict(zip(dictionary, nums))

In [13]:
x_vals = []

for user in tokenized_posts:
    x_vals.append([word_int[x] for x in user if x in word_int.keys()])

In [14]:
intro_extro = [word[0:1] for word in types[0:len(types)]]
bin_intro_extro = []
for letter in intro_extro:
    if (letter == 'I'):
        bin_intro_extro.append(0)
    else:
        bin_intro_extro.append(1)

In [44]:
import numpy as np
import random
random.seed(673)

x = np.array(x_vals)
random.shuffle(x)
test_data = x[:1500]
train_data = x[1500:]

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [45]:
from keras.utils.np_utils import to_categorical

y = np.asarray(bin_intro_extro).astype('float32')
random.shuffle(y)
y_test = y[:1500]
y_train = y[1500:]

In [46]:
x_val = x_train[:1500]
x_partial_train = x_train[1500:]

y_val = y_train[:1500]
y_partial_train = y_train[1500:]

In [47]:
from keras import models
from keras import layers
import tensorflow as tf

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape = (10000,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [57]:
model = build_model()
with tf.device('/gpu:0'):
    history = model.fit(x_partial_train, y_partial_train, epochs = 10, batch_size = 512, validation_data=(x_val, y_val), verbose=0)

In [ ]:
#K-Fold Validation
k = 4

# // is floor div operator
num_val_samples = len(x_train) // k

for num_epochs in range(1, 20):
    print('Epoch:', num_epochs)
    all_scores = []
    all_loss = []
    for i in range(k):
        print('processing fold #', i)
        val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples]
        val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

        partial_train_data = np.concatenate(
            [x_train[:i * num_val_samples],
             x_train[(i + 1) * num_val_samples:]],
            axis=0)
        partial_train_targets = np.concatenate(
            [y_train[:i * num_val_samples],
             y_train[(i + 1) * num_val_samples:]],
            axis=0)

        model = build_model()
        with tf.device('/gpu:0'):
            model.fit(partial_train_data, partial_train_targets,
                epochs=num_epochs, batch_size=64,verbose = 0)

            val_loss, val_acc = model.evaluate(val_data, val_targets, verbose = 0)

        all_loss.append(val_loss)
        all_scores.append(val_acc)

    print ("Accuracy Mean:", np.mean(all_scores))
    print ("Loss Mean:", np.mean(all_loss))

In [59]:
model = build_model()
with tf.device('/gpu:0'):
    model.fit(x_train, y_train, epochs=6, batch_size=512)
    
results = model.evaluate(x_test, y_test)
results

Epoch 1/6
7175/7175 [==============================] - 1s 185us/step - loss: 0.5584 - acc: 0.7660
Epoch 2/6
7175/7175 [==============================] - 1s 78us/step - loss: 0.5086 - acc: 0.7723
Epoch 3/6
7175/7175 [==============================] - 1s 78us/step - loss: 0.4817 - acc: 0.7723
Epoch 4/6
7175/7175 [==============================] - 1s 78us/step - loss: 0.4431 - acc: 0.7727
Epoch 5/6
7175/7175 [==============================] - 1s 79us/step - loss: 0.4183 - acc: 0.7728
Epoch 6/6
1500/1500 [==============================] - 0s 274us/step


[0.6820563006401063, 0.7566666669845581]